<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a932053af98bbe72779e00102e4181128e21809b07f42349579cb528960b949a
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-08 12:46:48
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15


-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.39 C
-------------------
Today PnL: -1.92 L (-1.36%)
Current PnL: -28.93 L (-18.69%)
CY Booked + Current PnL: -14.64 L (-9.45%)
-------------------
Total profit:  1.32 L
Total loss:  -30.25 L
-------------------
Total Booked + Current PnL: 12.34 L (9.66%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.25%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.56 L (67.14%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.86%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.22,-16.80,20.30,0.09,17021.0,-16930.0,83847.0,94.66,37.0,M-SC,2.38,253.0,-0.99,0.62,10.16,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-1.37,-13.44,15.54,0.02,23482.0,-23453.0,151106.0,130.47,49.0,M-SC,16.81,234.0,-1.00,1.12,31.44,OX40N,NTT,PAINTS
43,ITC,452.00,-0.43,-1.49,12.10,10.44,23857.0,-2973.0,197165.0,-41.15,46.0,X-LC,1.59,5.0,-0.12,1.47,4.00,X40,NTT,FMCG
50,MASFIN,398.61,-1.75,-3.55,26.54,22.05,25080.0,-3480.0,94500.0,-16.89,53.0,H-SC,6.86,164.0,-0.14,0.70,37.36,XR,ATH,FINANCE
35,ICICIGI,2252.93,-0.64,4.66,14.84,20.19,27075.0,8127.0,182447.0,-17.91,43.0,X-MC,6.16,68.0,0.30,1.36,20.48,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HCLTECH,1922.01,0.63,9.91,13.49,24.73,35869.0,23971.0,265895.0,14.88,72.0,X-LC,5.56,7.0,0.67,1.98,26.65,X40,ATH,IT
84,WIPRO,420.00,0.42,7.05,60.92,72.27,110506.0,11947.0,181395.0,-8.35,74.0,M-LC,2.80,99.0,0.11,1.35,13.66,XR,NTT,IT
13,BSOFT,831.70,0.38,-21.24,91.46,50.79,101312.0,-29881.0,110772.0,0.89,73.0,H-SC,5.61,151.0,-0.29,0.82,28.97,XR,ATH,IT
51,MEDANTA,1486.00,0.37,-2.49,27.25,24.08,33732.0,-3163.0,123787.0,-10.46,34.0,X-SC,5.84,89.0,-0.09,0.92,16.64,XY24,NTT,HEALTHCARE
73,TCS,4389.97,0.33,-11.72,35.12,19.28,107259.0,-40550.0,305406.0,-22.98,71.0,X-LC,6.04,1.0,-0.38,2.27,12.90,X40,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,KPIGREEN,730.66,-6.40,-20.27,84.30,46.95,84220.0,-25392.0,99905.0,-26.33,26.0,H-SC,4.25,154.0,-0.30,0.74,25.16,MH,ATH,POWER
18,COFFEEDAY,80.00,-3.72,-45.72,149.22,35.27,91967.0,-51917.0,61632.0,-57.75,22.0,L-SC,12.93,270.0,-0.56,0.46,50.85,XR,NTT,HOTELS
57,RAJOOENG,143.10,-3.69,-34.47,113.04,39.61,75929.0,-35330.0,67170.0,-55.65,26.0,H-SC,25.16,135.0,-0.47,0.50,0.00,AR,ATH,MISC
80,VALIANTORG,838.00,-3.66,-46.19,223.61,74.15,196873.0,-75562.0,88043.0,-258.45,38.0,H-SC,20.90,149.0,-0.38,0.66,12.86,XR,NTT,CHEMICALS
15,CAMS,4762.00,-3.24,-80.33,534.43,24.79,264757.0,-202316.0,49540.0,-81.32,40.0,X-SC,2.08,86.0,-0.76,0.37,21.11,X40N,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-2.10,-8.67,127.65,107.91,172131.0,-12798.0,134846.0,-25.82,25.0,M-SC,11.60,216.0,-0.07,1.00,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.37,-13.44,15.54,0.02,23482.0,-23453.0,151106.0,130.47,49.0,M-SC,16.81,234.0,-1.00,1.12,31.44,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.22,-16.80,20.30,0.09,17021.0,-16930.0,83847.0,94.66,37.0,M-SC,2.38,253.0,-0.99,0.62,10.16,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.74,-25.20,51.70,13.47,104288.0,-67950.0,201717.0,-69.28,27.0,H-SC,2.87,158.0,-0.65,1.50,2.57,XY24,NTT,PAINTS
66,SIS,528.00,-1.44,-25.52,62.81,21.26,51941.0,-28336.0,82696.0,1949.22,43.0,H-SC,4.25,166.0,-0.55,0.62,11.73,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.42,7.05,60.92,72.27,110506.0,11947.0,181395.0,-8.35,74.0,M-LC,2.8,99.0,0.11,1.35,13.66,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.42,7.05,60.92,72.27,110506.0,11947.0,181395.0,-8.35,74.0,M-LC,2.80,99.0,0.11,1.35,13.66,XR,NTT,IT
38,INDIAMART,4810.62,-2.35,-2.96,113.04,106.72,135285.0,-3657.0,119679.0,-51.96,33.0,H-SC,1.31,139.0,-0.03,0.89,19.72,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.52,-2.29,38.81,35.63,79475.0,-4795.0,204779.0,-15.85,38.0,H-MC,3.17,119.0,-0.06,1.52,14.00,AR,ATH,PHARMA
37,IEX,219.00,-2.22,-3.57,54.13,48.63,104371.0,-7138.0,192816.0,-35.77,49.0,H-SC,14.13,136.0,-0.07,1.43,7.57,XR,NTT,MISC
25,FINCABLES,1641.55,-2.10,-8.67,127.65,107.91,172131.0,-12798.0,134846.0,-25.82,25.0,M-SC,11.60,216.0,-0.07,1.00,0.00,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-2.08,-23.47,93.99,48.45,144216.0,-47068.0,153438.0,-28.89,10.0,X-MC,10.65,64.0,-0.33,1.14,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-1.73,-38.71,120.59,35.20,95106.0,-49803.0,78867.0,3.39,23.0,X-SC,15.04,92.0,-0.52,0.59,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.34,-12.00,38.17,21.58,47070.0,-16821.0,123318.0,-55.63,26.0,X-MC,5.34,56.0,-0.36,0.92,9.18,X40N,ATH,IT
46,JSWINFRA,342.00,-2.21,-11.29,29.89,15.23,53123.0,-22619.0,177728.0,-24.06,26.0,X-MC,7.56,66.0,-0.43,1.32,17.32,X40N,NTT,REALTY
54,PGHH,17907.65,-0.34,-6.30,42.76,33.76,80457.0,-12660.0,188160.0,-33.85,28.0,X-MC,5.07,57.0,-0.16,1.40,0.00,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,-0.95,-20.75,93.69,53.50,224043.0,-62619.0,239132.0,-61.59,34.0,X-MC,1.36,58.0,-0.28,1.78,6.21,XY24,NTT,FMCG
43,ITC,452.0,-0.43,-1.49,12.10,10.44,23857.0,-2973.0,197165.0,-41.15,46.0,X-LC,1.59,5.0,-0.12,1.47,4.00,X40,NTT,FMCG
3,ACC,3906.0,0.08,-24.37,117.05,64.15,210643.0,-57991.0,179960.0,-55.88,38.0,X-SC,1.68,82.0,-0.28,1.34,0.95,XY24,BTT,CEMENT
15,CAMS,4762.0,-3.24,-80.33,534.43,24.79,264757.0,-202316.0,49540.0,-81.32,40.0,X-SC,2.08,86.0,-0.76,0.37,21.11,X40N,NTT,MISC
36,ICICIPRULI,790.0,0.03,3.82,26.15,30.97,47983.0,6753.0,183491.0,-20.57,58.0,X-MC,2.21,75.0,0.14,1.37,16.86,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-1.73,-38.71,120.59,35.20,95106.0,-49803.0,78867.0,3.39,23.0,X-SC,15.04,92.0,-0.52,0.59,0.0,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-2.08,-23.47,93.99,48.45,144216.0,-47068.0,153438.0,-28.89,10.0,X-MC,10.65,64.0,-0.33,1.14,0.0,X40N,ATH,FINANCE
55,QUESS,424.00,-1.11,-32.76,112.42,42.84,49141.0,-21294.0,43712.0,-55.92,32.0,X-SC,37.31,83.0,-0.43,0.33,0.0,XY24,NTT,MISC
53,PAGEIND,51605.07,-0.81,-9.25,38.91,26.07,57820.0,-15140.0,148600.0,-33.14,30.0,X-MC,11.19,55.0,-0.26,1.11,0.0,X40,ATH,APPARELS
54,PGHH,17907.65,-0.34,-6.30,42.76,33.76,80457.0,-12660.0,188160.0,-33.85,28.0,X-MC,5.07,57.0,-0.16,1.40,0.0,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-1.11,-32.76,112.42,42.84,49141.0,-21294.0,43712.0,-55.92,32.0,X-SC,37.31,83.0,-0.43,0.33,0.00,XY24,NTT,MISC
15,CAMS,4762.00,-3.24,-80.33,534.43,24.79,264757.0,-202316.0,49540.0,-81.32,40.0,X-SC,2.08,86.0,-0.76,0.37,21.11,X40N,NTT,MISC
59,RELAXO,1176.00,-1.41,-48.35,194.88,52.31,146109.0,-70186.0,74974.0,-45.02,31.0,X-SC,5.76,91.0,-0.48,0.56,0.36,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.73,-38.71,120.59,35.20,95106.0,-49803.0,78867.0,3.39,23.0,X-SC,15.04,92.0,-0.52,0.59,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.34,-12.00,38.17,21.58,47070.0,-16821.0,123318.0,-55.63,26.0,X-MC,5.34,56.0,-0.36,0.92,9.18,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.33,-11.72,35.12,19.28,107259.0,-40550.0,305406.0,-22.98,71.0,X-LC,6.04,1.0,-0.38,2.27,12.90,X40,ATH,IT
41,INFY,2275.00,0.22,10.27,40.45,54.87,139559.0,32129.0,345017.0,-14.21,71.0,X-LC,3.40,2.0,0.23,2.57,19.40,X40,BTT,IT
75,TMPV,600.00,-0.95,-17.00,71.31,42.18,163595.0,-46996.0,229414.0,-25.52,28.0,X-LC,4.51,3.0,-0.29,1.71,0.00,XY24,NTT,AUTO
81,VBL,671.64,-2.78,-5.75,43.94,35.67,130805.0,-18151.0,297691.0,-17.02,49.0,X-LC,4.94,4.0,-0.14,2.21,7.31,X40N,ATH,FMCG
43,ITC,452.00,-0.43,-1.49,12.10,10.44,23857.0,-2973.0,197165.0,-41.15,46.0,X-LC,1.59,5.0,-0.12,1.47,4.00,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-2.97,-37.67,113.21,32.89,54479.0,-29084.0,48122.0,-694.37,54.0,L-MC,6.38,259.0,-0.53,0.36,32.52,XR,NTT,BANKS
6,ASIANTILES,137.00,-2.61,-19.37,125.33,81.67,94793.0,-18175.0,75635.0,6794.44,32.0,L-SC,15.48,271.0,-0.19,0.56,46.47,XR,NTT,CERAMICS
50,MASFIN,398.61,-1.75,-3.55,26.54,22.05,25080.0,-3480.0,94500.0,-16.89,53.0,H-SC,6.86,164.0,-0.14,0.70,37.36,XR,ATH,FINANCE
13,BSOFT,831.70,0.38,-21.24,91.46,50.79,101312.0,-29881.0,110772.0,0.89,73.0,H-SC,5.61,151.0,-0.29,0.82,28.97,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.37,-13.44,15.54,0.02,23482.0,-23453.0,151106.0,130.47,49.0,M-SC,16.81,234.0,-1.00,1.12,31.44,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.38,-21.24,91.46,50.79,101312.0,-29881.0,110772.0,0.89,73.0,H-SC,5.61,151.0,-0.29,0.82,28.97,XR,ATH,IT
5,ASIANPAINT,3460.25,-0.93,2.80,17.66,20.95,45705.0,7040.0,258808.0,1.76,73.0,X-LC,18.89,31.0,0.15,1.93,39.43,X40,ATH,PAINTS
32,HCLTECH,1922.01,0.63,9.91,13.49,24.73,35869.0,23971.0,265895.0,14.88,72.0,X-LC,5.56,7.0,0.67,1.98,26.65,X40,ATH,IT
84,WIPRO,420.00,0.42,7.05,60.92,72.27,110506.0,11947.0,181395.0,-8.35,74.0,M-LC,2.80,99.0,0.11,1.35,13.66,XR,NTT,IT
73,TCS,4389.97,0.33,-11.72,35.12,19.28,107259.0,-40550.0,305406.0,-22.98,71.0,X-LC,6.04,1.0,-0.38,2.27,12.90,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.89
1,25,45.85
2,50,76.74


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.79
MC    30.23
LC    24.95
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.29
X40      22.64
X40N     10.82
XR       10.00
XY25      9.39
AR        9.34
OX40N     7.74
SR        1.01
MH        0.74
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.36
X-MC    23.50
X-LC    21.36
M-SC    12.08
X-SC     6.97
H-MC     4.96
M-MC     1.41
L-SC     1.38
M-LC     1.35
H-LC     1.20
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.41,-7.50,42.26
IT,12.96,-15.71,78.26
FINANCE,9.54,-20.82,71.87
ELECTRICAL,6.25,-11.61,52.49
PAINTS,6.18,-11.11,27.62
MISC,6.12,-58.06,122.82
INSURANCE,4.58,-2.89,39.15
PHARMA,4.08,-1.75,34.32
AUTO,3.44,-24.12,77.33


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3233611.0,21
XR,1318435.0,13
AR,1313077.0,10
X40,998707.0,14
X40N,993104.0,9
OX40N,749890.0,10
XY25,380205.0,6
SR,284404.0,2
MH,84220.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3703006.0,25
M-SC,1431122.0,15
X-MC,1325291.0,16
X-SC,984666.0,8
X-LC,892974.0,11
H-MC,410871.0,3
L-SC,275511.0,3
M-LC,110506.0,1
H-LC,71694.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1298870.0      6
           AR         899360.0      5
M-SC       XY24       812439.0      6
H-SC       XR         791041.0      7
X-SC       X40N       506016.0      3
X-MC       X40        461324.0      7
           XY24       392633.0      3
X-LC       X40        352249.0      5
H-SC       OX40N      345111.0      4
M-SC       OX40N      316028.0      5
X-SC       XY24       293516.0      3
H-SC       SR         284404.0      2
X-MC       X40N       280842.0      4
X-LC       XY24       240299.0      2
H-MC       AR         215017.0      2
X-LC       X40N       206246.0      2
H-MC       XY24       195854.0      1
X-MC       XY25       190492.0      2
L-SC       XR         186760.0      2
X-SC       X40        185134.0      2
M-SC       XR         175649.0      2
           AR         127006.0      2
M-LC       XR         110506.0      1
X-LC       XY25        94180.0      2
L-SC       OX40N       88751.0      1
H-SC       MH          84220.0      1
H-LC       AR          71694.0      1
M-MC       XY25        56009.0      1
L-MC       XR          54479.0      1
L-LC       XY25        39524.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 39.0 seconds
